In [1]:
from geopandas import GeoDataFrame
from shapely.geometry import Point
import geemap

import os
import geopandas as gpd

from shapely import wkt
import pandas as pd
import matplotlib.pyplot as plt

### Funciones

In [2]:
def calcular_distancia(imagen_pixel, nom_vars_list, df_sh_list):
    
    """Funcion para calcular la distancia de la imagen con diferentes sectores (medido en metros)"""
    
    for i in range(len(df_sh_list)):
        print('revisando ' + str(i) + '----')
        df_sh_list[i].to_crs(epsg=32719, inplace=True)                # to meters
        imagen_pixel[nom_vars_list[i]] = imagen_pixel.geometry.apply(lambda x: df_sh_list[i].distance(x).min())  # add distance colums
    
    return imagen_pixel

def add_time(image):
    """Funcion para realizar un filtro de fechas con map"""
    return image.set('Date', image.date().format("YYYY-MM-dd"))



def calculate_weather(image, fecha, df_4326):
    
    # filtro df
    df_aux = df_4326[df_4326['time2'] == fecha] # global env
    
    # recolectamos los puntos
    points = df_aux.geometry

    # formato apto para buscar los puntos en gee
    jsonDict = eval(points.to_json()) 
    fc = ee.FeatureCollection(jsonDict)
    imagen_per_point = image.sampleRegions(collection = fc)
    
    fc_point_samp = geemap.ee_to_df(imagen_per_point)
    df_aux = df_aux.reset_index(drop= True)
    fc_point_samp = fc_point_samp.reset_index(drop= True)
    
    return pd.concat([df_aux, fc_point_samp], axis=1)


### Estimando distancias

In [3]:
muestra_datos_consolidados_v2 = pd.read_csv('muestra_datos_consolidados_v2.csv')

In [4]:
geometry = [Point(xy) for xy in zip(muestra_datos_consolidados_v2.x, muestra_datos_consolidados_v2.y)]
df = muestra_datos_consolidados_v2.drop(['x', 'y'], axis=1)
df = GeoDataFrame(df, crs="EPSG:32719", geometry=geometry)

In [5]:
df

,Unnamed: 0,time,dim_0,coastal,blue,green,red,nir08,swir16,swir22,...,codreg,codprov,codcom,nom_incen,causa,superficie,fecha_ini,fecha_ter,tipo_dato,geometry
0,0,2022-02-02,0,0.1473,0.1379,0.1561,0.1634,0.2745,0.3125,0.2441,...,5,56,5601,335-SAN JUAN DE HUINCA,1.9.1. Corte de cable eléctrico por caída de r...,310.116,2022-02-23,2022-03-02,zona_incendio,POINT (269030.000 6265610.000)
1,1,2022-02-02,1,0.1482,0.1382,0.1578,0.1847,0.2699,0.3430,0.2728,...,5,56,5601,335-SAN JUAN DE HUINCA,1.9.1. Corte de cable eléctrico por caída de r...,310.116,2022-02-23,2022-03-02,zona_incendio,POINT (271190.000 6265270.000)
2,2,2022-02-02,2,0.1333,0.1437,0.1606,0.1840,0.2988,0.3306,0.2551,...,5,56,5601,335-SAN JUAN DE HUINCA,1.9.1. Corte de cable eléctrico por caída de r...,310.116,2022-02-23,2022-03-02,zona_incendio,POINT (271570.000 6264370.000)
3,3,2022-02-02,3,0.1305,0.1355,0.1490,0.1665,0.2765,0.2990,0.2256,...,5,56,5601,335-SAN JUAN DE HUINCA,1.9.1. Corte de cable eléctrico por caída de r...,310.116,2022-02-23,2022-03-02,zona_incendio,POINT (271310.000 6264570.000)
4,4,2022-02-02,4,0.1516,0.1521,0.1757,0.2066,0.3199,0.4175,0.3203,...,5,56,5601,335-SAN JUAN DE HUINCA,1.9.1. Corte de cable eléctrico por caída de r...,310.116,2022-02-23,2022-03-02,zona_incendio,POINT (267970.000 6264290.000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2996,1696,2021-08-01,294,0.8019,0.7227,0.7118,0.7175,0.6358,0.0299,0.0296,...,5,54,5401,8-FUNDO LA RETAMILLA,2.1.11. Otros intencionales no clasificados,210.938,2021-07-25,2021-08-09,zona_random,POINT (386050.000 6404410.000)
2997,1697,2021-08-01,295,0.5904,0.7795,0.7561,0.7404,0.6300,0.0209,0.0228,...,5,54,5401,8-FUNDO LA RETAMILLA,2.1.11. Otros intencionales no clasificados,210.938,2021-07-25,2021-08-09,zona_random,POINT (390390.000 6384990.000)
2998,1698,2021-08-01,297,0.0103,0.0126,0.0219,0.0326,0.0772,0.1753,0.1311,...,5,54,5401,8-FUNDO LA RETAMILLA,2.1.11. Otros intencionales no clasificados,210.938,2021-07-25,2021-08-09,zona_random,POINT (287090.000 6311990.000)
2999,1699,2021-08-01,298,0.0270,0.0477,0.0609,0.0771,0.1695,0.2524,0.1927,...,5,54,5401,8-FUNDO LA RETAMILLA,2.1.11. Otros intencionales no clasificados,210.938,2021-07-25,2021-08-09,zona_random,POINT (279230.000 6404450.000)


In [6]:
# load data pixeles
# df = pd.read_csv('muestra_datos_consolidados_v2.csv')

# # editando geometria
# df['geometry'] = df['geometry'].apply(wkt.loads)
# df = gpd.GeoDataFrame(df, geometry='geometry')
# df.crs = "EPSG:32719"

In [7]:
# load .shp
lista_gdfs = []

for directorio_raiz, directorios, archivos in os.walk('shp'):
    for archivo in archivos:
        if archivo.endswith('.shp'):            # Solo considerar archivos con extensión .shp
            ruta_archivo = os.path.join(directorio_raiz, archivo)
            print(ruta_archivo)
            gdf = gpd.read_file(ruta_archivo)   # Cargar el Shapefile
            lista_gdfs.append(gdf)   

shp\areas_silverstres_protegidas\snaspe.shp
shp\areas_urbanas\Areas_Pobladas.shp
shp\masas_lacustres\masas_lacustres.shp
shp\Masas_lacustres_sector_sur\Masas_lacustres_sector_sur.shp
shp\red_ferroviaria\Red_ferroviaria.shp
shp\red_hidrografica\Red_Hidrografica.shp
shp\red_vial\redvial2019.shp


NO SIRVEN:

- shp\masas_lacustres\masas_lacustres.shp
- shp\red_ferroviaria\Red_ferroviaria.shp 


In [8]:
areas_silvestres_protegidas = lista_gdfs[0][lista_gdfs[0]['Region'] == 'Región de Valparaíso']
red_hidrografica = lista_gdfs[5][lista_gdfs[5]['Region'] == 'Región de Valparaíso']

In [9]:
# AREAS URBANAS VALPO

aux = red_hidrografica['Comuna'].unique()  # no esta rapa nui ni juan fernandez!
aux[1] = 'La Calera'

areas_urbanas = lista_gdfs[1][lista_gdfs[1]['comuna'].isin(aux)]

In [10]:
red_vial_2019 = lista_gdfs[6][lista_gdfs[6]['Cod_Region']==5]

#### Calculando distancias:

In [11]:
nom_vars_list = ['d_areas_silvestres_protegidas', 'd_red_hidrografica', 'd_areas_urbanas', 'd_red_vial', 'd_masas_lacustres_sur']
df_sh_list = [areas_silvestres_protegidas, red_hidrografica, areas_urbanas, red_vial_2019, lista_gdfs[3]]

df_dist = calcular_distancia(df, nom_vars_list, df_sh_list)
df_dist

revisando 0----


c:\Users\jmpreussa\Desktop\dataton\.venv\lib\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


revisando 1----


c:\Users\jmpreussa\Desktop\dataton\.venv\lib\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


revisando 2----


c:\Users\jmpreussa\Desktop\dataton\.venv\lib\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


revisando 3----


c:\Users\jmpreussa\Desktop\dataton\.venv\lib\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


revisando 4----


,Unnamed: 0,time,dim_0,coastal,blue,green,red,nir08,swir16,swir22,...,superficie,fecha_ini,fecha_ter,tipo_dato,geometry,d_areas_silvestres_protegidas,d_red_hidrografica,d_areas_urbanas,d_red_vial,d_masas_lacustres_sur
0,0,2022-02-02,0,0.1473,0.1379,0.1561,0.1634,0.2745,0.3125,0.2441,...,310.116,2022-02-23,2022-03-02,zona_incendio,POINT (269030.000 6265610.000),16339.404583,758.594353,12042.532160,2851.080110,1852.941497
1,1,2022-02-02,1,0.1482,0.1382,0.1578,0.1847,0.2699,0.3430,0.2728,...,310.116,2022-02-23,2022-03-02,zona_incendio,POINT (271190.000 6265270.000),18310.249301,361.717646,13811.511319,1691.925296,3350.347742
2,2,2022-02-02,2,0.1333,0.1437,0.1606,0.1840,0.2988,0.3306,0.2551,...,310.116,2022-02-23,2022-03-02,zona_incendio,POINT (271570.000 6264370.000),18464.842794,407.545429,14703.240908,801.074859,3175.668222
3,3,2022-02-02,3,0.1305,0.1355,0.1490,0.1665,0.2765,0.2990,0.2256,...,310.116,2022-02-23,2022-03-02,zona_incendio,POINT (271310.000 6264570.000),18255.076467,169.431644,14377.193643,1127.997228,3038.104378
4,4,2022-02-02,4,0.1516,0.1521,0.1757,0.2066,0.3199,0.4175,0.3203,...,310.116,2022-02-23,2022-03-02,zona_incendio,POINT (267970.000 6264290.000),14946.101288,258.781302,12387.794590,4270.413057,227.462127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2996,1696,2021-08-01,294,0.8019,0.7227,0.7118,0.7175,0.6358,0.0299,0.0296,...,210.938,2021-07-25,2021-08-09,zona_random,POINT (386050.000 6404410.000),90270.225200,1014.828177,47001.213759,34642.803276,127651.479892
2997,1697,2021-08-01,295,0.5904,0.7795,0.7561,0.7404,0.6300,0.0209,0.0228,...,210.938,2021-07-25,2021-08-09,zona_random,POINT (390390.000 6384990.000),84605.918995,165.709330,29532.160980,19090.661756,122273.038222
2998,1698,2021-08-01,297,0.0103,0.0126,0.0219,0.0326,0.0772,0.1753,0.1311,...,210.938,2021-07-25,2021-08-09,zona_random,POINT (287090.000 6311990.000),15746.639782,402.528328,9743.070144,1138.049730,3099.419300
2999,1699,2021-08-01,298,0.0270,0.0477,0.0609,0.0771,0.1695,0.2524,0.1927,...,210.938,2021-07-25,2021-08-09,zona_random,POINT (279230.000 6404450.000),15786.220912,306.051817,4513.828403,1543.146811,50710.357500


In [12]:
del lista_gdfs
df_dist.describe()

,Unnamed: 0,dim_0,coastal,blue,green,red,nir08,swir16,swir22,spatial_ref,...,id_pixel_incendio,codreg,codprov,codcom,superficie,d_areas_silvestres_protegidas,d_red_hidrografica,d_areas_urbanas,d_red_vial,d_masas_lacustres_sur
count,3001.000000,3001.000000,2807.000000,2804.000000,2804.000000,2804.000000,2807.000000,2810.000000,2807.000000,3001.0,...,3001.000000,3001.0,3001.000000,3001.000000,3001.000000,3001.000000,3001.000000,3001.000000,3001.000000,3001.000000
mean,763.145618,104.462512,0.105684,0.110628,0.126444,0.144505,0.214676,0.235138,0.192849,32719.0,...,116.078640,5.0,55.040320,5505.031989,262.526423,32063.992534,552.112020,12730.133436,4623.680606,43448.352947
std,455.647673,84.458294,0.133021,0.128256,0.127431,0.129575,0.124578,0.115020,0.101135,0.0,...,66.919806,0.0,0.999353,99.935334,49.556933,23319.440369,468.566666,8344.899854,5898.813183,39137.152390
min,0.000000,0.000000,0.000000,0.000100,0.000100,0.000100,0.000100,0.000100,0.003000,32719.0,...,1.000000,5.0,54.000000,5401.000000,210.938000,0.000000,0.389959,0.000000,13.229785,115.120424
25%,375.000000,39.000000,0.024250,0.032300,0.046800,0.059475,0.129600,0.156650,0.121350,32719.0,...,58.000000,5.0,54.000000,5401.000000,210.938000,16567.128185,212.795014,9218.933064,1323.754437,2984.193250
50%,750.000000,77.000000,0.058900,0.068900,0.091550,0.117200,0.194600,0.226500,0.185300,32719.0,...,116.000000,5.0,56.000000,5601.000000,310.116000,26399.830430,451.854555,11189.664645,3179.861715,47475.396285
75%,1125.000000,162.000000,0.146700,0.155025,0.173950,0.195325,0.283500,0.314400,0.260450,32719.0,...,174.000000,5.0,56.000000,5601.000000,310.116000,36924.204212,767.098830,14060.238328,4989.218681,68115.419125
max,1700.000000,299.000000,0.983600,0.994400,0.980400,0.984600,0.938300,0.822800,0.799100,32719.0,...,240.000000,5.0,56.000000,5601.000000,310.116000,98272.973758,4020.023266,49738.861969,36704.684242,132180.754843


### Datos climáticos con Google Earth Engine

In [13]:
import ee
ee.Authenticate()

True

In [14]:
# Inicializar Earth Engine
ee.Initialize()

In [15]:
df_4326 = df.to_crs("EPSG:4326")
df_4326['time2'] = [item.split(" ", 1)[0] for item in df_4326['time']]

# Obtenemos las fechas de captura de cada uno, para realizar un posterior filtro en las imagenes recolectadas
DATES = df_4326.time.unique()
DATES = [item.split(" ", 1)[0] for item in DATES]
DATES

inicio_datetime = min(DATES)
fin_datetime = max(DATES)

In [16]:
list_df = []

for fecha in DATES:
    print('Extrayendo info para el día: ' + fecha)
    
    # Definir la colección de imágenes
    image_collection = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR') \
        .filterDate(fecha) \
        .select([
            'temperature_2m',
            'temperature_2m_min',
            'temperature_2m_max',
            'dewpoint_temperature_2m',
            'dewpoint_temperature_2m_min',
            'dewpoint_temperature_2m_max',
            'soil_temperature_level_1',
            'soil_temperature_level_1_min',
            'soil_temperature_level_1_max',
            'volumetric_soil_water_layer_1',
            'volumetric_soil_water_layer_1_min',
            'volumetric_soil_water_layer_1_max',
            'u_component_of_wind_10m',
            'u_component_of_wind_10m_min',
            'u_component_of_wind_10m_max',
            'v_component_of_wind_10m',
            'v_component_of_wind_10m_min',
            'v_component_of_wind_10m_max',
            'total_precipitation_sum',
            'total_precipitation_min',
            'total_precipitation_max'
        ])
        
    # Realizando estimaciones: 
    df_aux = calculate_weather(image_collection.first(), fecha, df_4326)
              
    list_df.append(df_aux)
    
data_con_gee = pd.concat(list_df, ignore_index = True)
        

Extrayendo info para el día: 2022-02-02
Extrayendo info para el día: 2022-02-07
Extrayendo info para el día: 2022-02-12
Extrayendo info para el día: 2022-02-17
Extrayendo info para el día: 2022-02-22
Extrayendo info para el día: 2022-02-27
Extrayendo info para el día: 2022-03-04
Extrayendo info para el día: 2021-07-02
Extrayendo info para el día: 2021-07-07
Extrayendo info para el día: 2021-07-17
Extrayendo info para el día: 2021-07-22
Extrayendo info para el día: 2021-07-27
Extrayendo info para el día: 2021-08-01


In [17]:
data_con_gee.to_csv('data_con_geee.csv')